In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from math import log2
import plotly.graph_objects as go

from words_class import Words, WORDS

In [2]:
from frequency import *

In [3]:
dg = pd.read_csv('genele-info-dataframe.csv')

In [4]:
dg.head()

,symbol,len,distinct,padded_symbol,char1,char2,char3,char4,char5,freq_char,freq_char_dist,freq_symbol,freq_symbol_dist,freq_pos
0,CAP1,5,5,CAP1,C,A,P,1,,0.325680,0.325680,0.296130,0.296130,0.155405
1,PRC1,5,5,PRC1,P,R,C,1,,0.323959,0.323959,0.296652,0.296652,0.152118
2,RRP1,5,4,RRP1,R,R,P,1,,0.322237,0.333746,0.297922,0.306896,0.149752
3,CCR1,5,4,CCR1,C,C,R,1,,0.314706,0.322180,0.287886,0.295939,0.149717
4,SRA1,5,5,SRA1,S,R,A,1,,0.300078,0.300078,0.277415,0.277415,0.149665


In [7]:
W = Words(dg['padded_symbol'])
#W.update('cap1','10000')
#W.update('scrt2','02001')
#W.update('lce2d','02020')
#W.update('hcg24','22220')
#W.update('gad1','02122')
#W.update('vcx','22022')

dg_ = W.table.copy()
dg_.sort_values('freq_pos',ascending=False)

,word0,word,num_distinct,char0,char1,char2,char3,char4,freq_char,freq_char_dist,freq_symb,freq_symb_dist,freq_pos
0,CAP1,CAP1,5,C,A,P,1,,18725,18725,17026,17026,8935
1,PRC1,PRC1,5,P,R,C,1,,18626,18626,17056,17056,8746
2,RRP1,RRP1,4,R,R,P,1,,18527,15351,17129,14116,8610
3,CCR1,CCR1,4,C,C,R,1,,18094,14819,16552,13612,8608
4,SRA1,SRA1,5,S,R,A,1,,17253,17253,15950,15950,8605
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11494,YWHAG,YWHAG,5,Y,W,H,A,G,7018,7018,6600,6600,959
11495,YWHAH,YWHAH,4,Y,W,H,A,H,6764,5415,6365,5061,953
11496,YWHAE,YWHAE,5,Y,W,H,A,E,6898,6898,6499,6499,949
11497,YWHAZ,YWHAZ,5,Y,W,H,A,Z,5737,5737,5378,5378,922


In [46]:
bad_chars = ''.upper() ###
good_chars = ''.upper()
char1_wrong_pos = ''.upper()
char2_wrong_pos = ''.upper()
char3_wrong_pos = ''.upper()
char4_wrong_pos = ''.upper()
char5_wrong_pos = ''.upper()
good_but_not_repeated = ''.upper()


dg_ = dg[
#    (dg['len']==3) &
    ~(dg['padded_symbol'].str.contains('|'.join('='+bad_chars).upper())) &
    (dg['padded_symbol'].apply(lambda s: all([x in s for x in good_chars]))) &
#    (dg['char1']=='r'.upper()) &
#    (dg['char2']=='a'.upper()) &
#    (dg['char3']=='f'.upper()) &
#    (dg['char4']=='1'.upper()) &
#    (dg['char4']==' '.upper()) &
#    (dg['char5']==' '.upper()) &
#    (dg['padded_symbol'].str.count('r'.upper())<2) &
    True
].copy()


freq_char = char_frequency(dg_['padded_symbol'])
freq_symb = symbol_frequency(dg_['padded_symbol'])
freq_pos = position_frequency(dg_['padded_symbol'])
words = list(dg_['padded_symbol'])


dg_['freq_char'] = dg_['padded_symbol'].apply(lambda x:freq_score_sum(x,frequency=freq_char))
dg_['freq_char_dist'] = dg_['padded_symbol'].apply(lambda x:freq_score_sum_distinct(x,frequency=freq_char))
dg_['freq_symbol'] = dg_['padded_symbol'].apply(lambda x:freq_score_sum(x,frequency=freq_symb))
dg_['freq_symbol_dist'] = dg_['padded_symbol'].apply(lambda x:freq_score_sum_distinct(x,frequency=freq_symb))
dg_['freq_pos'] = dg_['padded_symbol'].apply(lambda x: freq_score_sum_pos(x,frequency=freq_pos))
dg_ = dg_.sort_values('freq_symbol_dist',ascending=0)
dg_

,symbol,len,distinct,padded_symbol,char1,char2,char3,char4,char5,freq_char,freq_char_dist,freq_symbol,freq_symbol_dist,freq_pos
118,RPA1,5,5,RPA1,R,P,A,1,,18626,18626,17099,17099,8025
1,PRC1,5,5,PRC1,P,R,C,1,,18626,18626,17056,17056,8746
0,CAP1,5,5,CAP1,C,A,P,1,,18725,18725,17026,17026,8935
10,CPA1,5,5,CPA1,C,P,A,1,,18725,18725,17026,17026,8526
36,ACP1,5,5,ACP1,A,C,P,1,,18725,18725,17026,17026,8344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11461,ZMYM5,5,4,ZMYM5,Z,M,Y,M,5,5649,3616,5394,3478,1411
10161,TTTY7,5,3,TTTY7,T,T,T,Y,7,8791,3601,8316,3434,2475
10176,TTTY8,5,3,TTTY8,T,T,T,Y,8,8771,3581,8293,3411,2469
11471,ZMYM6,5,4,ZMYM6,Z,M,Y,M,6,5497,3464,5245,3329,1333


In [49]:
answer = ''.ljust(5).upper()

fig = go.Figure(layout_height=500, layout_width=750)
for f in ['freq_char','freq_symbol','freq_pos']:
    fig.add_trace(go.Violin(name=f, x=dg[f], marker_color='darkblue', showlegend=False))
    fig.add_trace(go.Scatter(x=[dg[dg['padded_symbol']==answer][f].values[0]], y=[f], marker_color='red', marker_size=15, showlegend=False))
fig.show()

IndexError: index 0 is out of bounds for axis 0 with size 0